In [ ]:
# Defining head, discharge and power values for the vertices
psh.head_discharge_limits = pd.DataFrame(columns=psh.hydro_units, index=['max', 'min'])
psh.limhead_discharge_limitsits.loc['max', 'head'] = 10
psh.head_discharge_limits.loc['min', 'head'] = 4
psh.head_discharge_limits.loc['max', 'discharge'] = 30
psh.head_discharge_limits.loc['min', 'discharge'] = 15

efficieny = 0.98

head_interval = (psh.max_head - psh.min_head) / (len(psh.subregions) / 2)
discharge_interval = (psh.max_discharge - psh.min_discharge) / len(psh.subregions)

h_vertices = np.zeros((2,2,10,3))
q_vertices = np.zeros((2,2,10,3))

# head vertices
for i in cascaded_reservoirs:
    for j in hydro_units[i]:
        for l in subregions:
            if l % 2 == 0:
                h_vertices[i,j,l,0] = min_head + l / 2 * head_interval # bottom left
                h_vertices[i,j,l,1] = min_head + l / 2 * head_interval + head_interval # bottom right
                h_vertices[i,j,l,2] = h_vertices[i,j,l,0] # top left

                h_vertices[i,j,l+1,0] = h_vertices[i,j,l,0] # top left
                h_vertices[i,j,l+1,1] = h_vertices[i,j,l,1]# top right
                h_vertices[i,j,l+1,2] = h_vertices[i,j,l,1] # bottom right

# discharge vertices
for i in cascaded_reservoirs:
    for j in hydro_units[i]:
        for l in subregions:
            if l % 2 == 0:
                q_vertices[i,j,l,0] = min_discharge # bottom left
                q_vertices[i,j,l,1] = min_discharge # bottom right
                q_vertices[i,j,l,2] = max_discharge # top left

                q_vertices[i,j,l+1,0] = max_discharge # top left
                q_vertices[i,j,l+1,1] = max_discharge # top right
                q_vertices[i,j,l+1,2] = min_discharge # bottom right

In [ ]:
power_vertices = np.zeros((2,2,10,3))

for i in cascaded_reservoirs:
    for j in hydro_units[i]:
        for l in subregions:
            for r in vertices:
                power_vertices[i,j,l,r] = production_function(efficieny, h_vertices[i,j,l,r], q_vertices[i,j,l,r])

In [ ]:
psh.clear()
volume = psh.continuous_var_matrix(psh.reservoirs, psh.periods, name='reservoir_volume')
total_discharge = psh.continuous_var_matrix(psh.reservoirs, psh.periods, name='total_discharge_ex_ps')
hu_discharge = psh.continuous_var_cube(psh.reservoirs, psh.hydro_units, psh.periods, name='discharge_hydro_units')
hu_head = psh.continuous_var_cube(psh.reservoirs, psh.hydro_units, psh.periods, name='head_hydro_units')
hu_power = psh.continuous_var_cube(psh.reservoirs, psh.hydro_units, psh.periods, name='power_output_hydro_units')
active_subregion = psh.binary_var_dict([(i, j, t, l) for i in psh.reservoirs for j in psh.hydro_units for t in psh.periods for l in psh.subregions], name='active_subregions')
alpha_coefficient = psh.continuous_var_dict([(i, j, t, l, r) for i in psh.reservoirs for j in psh.hydro_units for t in psh.periods for l in psh.subregions for r in psh.vertices], name='convex_combination_coefficients')
hu_start_up = psh.binary_var_cube(psh.reservoirs, psh.hydro_units, psh.periods, name='start_ups_hydro_units')
hu_shut_down = psh.binary_var_cube(psh.reservoirs, psh.hydro_units, psh.periods, name='shut_downs_hydro_units')
hu_state = psh.binary_var_cube(psh.reservoirs, psh.hydro_units, psh.periods, name='state_hydro_units')
ps_power = psh.continuous_var_cube(psh.reservoirs, psh.pumped_storage_units, psh.periods, name='total_power_output_pumped_storage_units')
ps_discharge = psh.continuous_var_cube(psh.reservoirs, psh.pumped_storage_units, psh.periods, name='total_discharge_pumped_storage_units')
ps_power_gen = psh.continuous_var_cube(psh.reservoirs, psh.pumped_storage_units, psh.periods, name='power_output_pumped_storage_units')
ps_power_pump = psh.continuous_var_cube(psh.reservoirs, psh.pumped_storage_units, psh.periods, name='power_consumption_pumped_storage_units')
ps_discharge_gen = psh.continuous_var_cube(psh.reservoirs, psh.pumped_storage_units, psh.periods, name='discharge_pumped_storage_units')
ps_discharge_pump = psh.continuous_var_cube(psh.reservoirs, psh.pumped_storage_units, psh.periods, name='pumped_upstream_pumped_storage_units')
ps_state_gen = psh.binary_var_cube(psh.reservoirs, psh.pumped_storage_units, psh.periods, name='generation_state_pumped_storage_units')
ps_state_pump = psh.binary_var_cube(psh.reservoirs, psh.pumped_storage_units, psh.periods, name='pumping_state_pumped_storage_units')

In [ ]:
# def __init__(self, reservoirs=None, upstream_reservoirs=None, hydro_units=None, hydro_unit_affiliation=None, pumped_storage_units=None, pumped_storage_unit_affiliation=None, periods=None, subregions=None, vertices=None, initial_volume=None, hu_initial_state=None, projected_prices=None, period_length=None, natural_inflow=None, travel_times=None, spillage=None, volume_limits=None, ps_power_generation_limits=None, ps_power_pumping_limits=None, ps_discharge_generation_limits=None, ps_discharge_pumping_limits=None, head_limits=None, discharge_limits=None, h_vertices=None, q_vertices=None, p_vertices=None, rho_values=None, volume=None, total_discharge=None, hu_discharge=None, hu_head=None, hu_power=None, active_subregion=None, alpha_coefficient=None, hu_start_up=None, hu_shut_down=None, hu_state=None, ps_power=None, ps_discharge=None, ps_power_gen=None, ps_power_pump=None, ps_discharge_gen=None, ps_discharge_pump=None, ps_state_gen=None, ps_state_pump=None):
    #     super().__init__()

    #     # Indices
    #     self.reservoirs = []
    #     self.upstream_reservoirs = []
    #     self.hydro_units = []
    #     self.hydro_unit_affiliation = {}
    #     self.pumped_storage_unit_affiliation = {}
    #     self.pumped_storage_units = []
    #     self.periods = []
    #     self.subregions = []
    #     self.vertices = [1,2,3]

    #     # Parameters
    #     self.initial_volume = 0
    #     self.hu_initial_state = {}
    #     self.projected_prices = []
    #     self.period_length = 0
    #     self.natural_inflow = []
    #     self.travel_times = pd.DataFrame()
    #     self.spillage = []
    #     self.volume_limits = pd.DataFrame()
    #     self.ps_power_generation_limits = pd.DataFrame()
    #     self.ps_power_pumping_limits = pd.DataFrame()
    #     self.ps_discharge_generation_limits = pd.DataFrame()
    #     self.ps_discharge_pumping_limits = pd.DataFrame()

    #     # Linearization
    #     self.head_limits = pd.DataFrame()
    #     self.discharge_limits = pd.DataFrame()
    #     self.h_vertices = []
    #     self.q_vertices = []
    #     self.p_vertices = []
    #     self.rho_values = pd.DataFrame()

In [ ]:
# Creating a model instance
psh = PSH()

# Indices
psh.reservoirs = np.arange(2)
psh.upstream_reservoirs = {
    0: [],
    1: [0]
}
psh.hydro_units = np.arange(4)
psh.hydro_unit_affiliation = {
    0: [0,1],
    1: [2,3]
}
psh.pumped_storage_units = np.arange(2)
psh.pumped_storage_unit_affiliation = {
    0: [0,1],
    1: []
}
psh.periods = np.arange(2)
psh.subregions = np.arange(10)
psh.vertices = np.arange(3)

# Parameters
psh.initial_volume = 20
psh.hu_initial_state = {
    0: 0,
    1: 1,
    2: 0,
    3: 1
}
psh.projected_prices = [10, 20, 15, 15, 15, 10, 10, 10, 5, 25]
psh.period_length = 1
psh.natural_inflow = np.ones(shape=(2,10)) * 2
psh.travel_times = pd.DataFrame(data=[[0,1],[0,0]], index=psh.reservoirs, columns=psh.reservoirs)
psh.spillage = np.ones(shape=(2,10))

# Reservoir volume limits
psh.volume_limits = pd.DataFrame(index=['max', 'min'], columns=psh.reservoirs)
psh.volume_limits.loc['max':,] = 20
psh.volume_limits.loc['min':,] = 10

# Pumped-storage unit limits
psh.ps_power_generation_limits = pd.DataFrame(index=['max', 'min'], columns=psh.pumped_storage_units)
psh.ps_power_generation_limits.loc['max':,] = 8
psh.ps_power_generation_limits.loc['min':,] = 2

psh.ps_power_pumping_limits = pd.DataFrame(index=['max', 'min'], columns=psh.pumped_storage_units)
psh.ps_power_pumping_limits.loc['max':,] = 4
psh.ps_power_pumping_limits.loc['min':,] = 1

psh.ps_discharge_generation_limits = pd.DataFrame(index=['max', 'min'], columns=psh.pumped_storage_units)
psh.ps_discharge_generation_limits.loc['max':,] = 6
psh.ps_discharge_generation_limits.loc['min':,] = 2

psh.ps_discharge_pumping_limits = pd.DataFrame(index=['max', 'min'], columns=psh.pumped_storage_units)
psh.ps_discharge_pumping_limits.loc['max':,] = 4
psh.ps_discharge_pumping_limits.loc['min':,] = 1

# Hydro unit head and discharge limits
psh.head_limits = pd.DataFrame(index=['max', 'min'], columns=psh.hydro_units)
psh.head_limits.loc['max':,] = 10
psh.head_limits.loc['min':,] = 4

psh.discharge_limits = pd.DataFrame(index=['max', 'min'], columns=psh.hydro_units)
psh.discharge_limits.loc['max':,] = 20
psh.discharge_limits.loc['min':,] = 10

# Defining the rho-values to calculate the power production
psh.rho_values = pd.DataFrame(index=np.arange(6), columns=psh.hydro_units)
psh.rho_values.loc[0:,] = 0.85
psh.rho_values.loc[1:,] = 0.02
psh.rho_values.loc[2:,] = 0.03
psh.rho_values.loc[3:,] = -0.0001
psh.rho_values.loc[4:,] = -0.0002
psh.rho_values.loc[5:,] = -0.00001

# Defining the head, discharge and power vertices for the linearization
psh.h_vertices = psh.head_vertices()
psh.q_vertices = psh.discharge_vertices()
psh.p_vertices = psh.power_vertices()